In [21]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [22]:
'''
tw_mitaka_a = pd.read_csv("tw_mitaka_a.csv")
tw_mitaka_1 = pd.read_csv("tw_mitaka1.csv")
x = pd.concat([tw_mitaka_a, tw_mitaka_1]).drop_duplicates()
x.to_csv('tw_mitaka1.csv', index=False)
'''

'\ntw_mitaka_a = pd.read_csv("tw_mitaka_a.csv")\ntw_mitaka_1 = pd.read_csv("tw_mitaka1.csv")\nx = pd.concat([tw_mitaka_a, tw_mitaka_1]).drop_duplicates()\nx.to_csv(\'tw_mitaka1.csv\', index=False)\n'

In [48]:
tw_mitaka = pd.read_csv("tw_mitaka1.csv")
tw_mitaka['created_at'] =  pd.to_datetime(tw_mitaka['created_at'])

In [49]:
tw_mitaka.shape

(1285, 7)

In [51]:
tw_mitaka.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285 entries, 0 to 1284
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tweet_id    1285 non-null   int64              
 1   username    1285 non-null   object             
 2   text        1285 non-null   object             
 3   created_at  1285 non-null   datetime64[ns, UTC]
 4   location    827 non-null    object             
 5   source      1285 non-null   object             
 6   url         1285 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(1), object(5)
memory usage: 70.4+ KB


In [52]:
tw_mitaka.head()

,tweet_id,username,text,created_at,location,source,url
0,1556927525313323009,denpochi,"hey, could u stop posting about mitaka? i ship...",2022-08-09 08:57:03+00:00,17 they/them\ncsm bllk jjk opm\nic kamonegioisi,Twitter for Android,https://twitter.com/denpochi/status/1556927525...
1,1556917447805898754,Lizsa___,join friend's style challenge\nAsa Mitaka from...,2022-08-09 08:17:00+00:00,NaN,Twitter for Android,https://twitter.com/Lizsa___/status/1556917447...
2,1556905368520998913,politicfemboy,I desire the same as Asa Mitaka... https://t.c...,2022-08-09 07:29:00+00:00,NaN,Twitter Web App,https://twitter.com/politicfemboy/status/15569...
3,1556888261964214272,killmekirby,why does my asa mitaka drawing look like yumek...,2022-08-09 06:21:01+00:00,she/her • 17,Twitter for iPhone,https://twitter.com/killmekirby/status/1556888...
4,1556884584977006592,retromanjapan,The Showa Retro In #MITAKA #Tokyo #8: A One-St...,2022-08-09 06:06:25+00:00,"Tokyo, Japan",Revive Social App,https://twitter.com/retromanjapan/status/15568...


In [54]:
#Stop words, links, emotes, non-ascii characters are removed.
stops = set(stopwords.words('english'))

for i in range(len(tw_mitaka['text'])):
    tweet_text = []
    #https://stackoverflow.com/a/24399874
    tw_mitaka['text'][i] = re.sub(r'[^\x00-\x7f]',r'', tw_mitaka['text'][i])
    #https://stackoverflow.com/a/65330690
    tw_mitaka['text'][i] = re.sub(r"http\S+", "", tw_mitaka['text'][i])
    
    words = tw_mitaka['text'][i].lower().split()
    words = [''.join(letter for letter in word if letter not in (string.punctuation)) for word in words]
    for word in words:
        if word not in stops:
            tweet_text.append(word)

    tw_mitaka['text'].iloc[i] = ' '.join(tweet_text)

In [56]:
print(tw_mitaka['text'][25])

['vomit', 'nuclear', 'weapons', 'chainsawman', 'procreate', 'mitaka']


In [63]:
#Word frequency dict
word_freq = {}

for text_list in tw_mitaka['text']:
    for word in text_list.split():
        if word not in word_freq:
            word_freq[word] = 0
        word_freq[word] += 1

In [67]:
dict(list(word_freq.items())[0:5])

{'hey': 5, 'could': 11, 'u': 17, 'stop': 5, 'posting': 2}

In [70]:
#Creating dataframe from word freq dict
df_wordfreq = pd.DataFrame.from_dict(word_freq, orient='index', columns=['Freq'])

In [72]:
df_wordfreq.sort_values('Freq', ascending=False).head(5)

,Freq
devil,685
war,679
mitaka,542
chainsawman,339
asa,273


#### VADER Sentimental Analysis

In [102]:
#Sentimental analysis function with VADER
def sentiment_scores(sentence):
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    if sentiment_dict['compound'] >= 0.05 :
        return("Positive")
    elif sentiment_dict['compound'] <= -0.05 :
        return("Negative")
    else :
        return("Neutral")

In [165]:
#Adding score column for sentiment analysis
tw_mitaka_vader = tw_mitaka.copy()

In [167]:
#VADER Sentimental analysis on text. 
score_list = []
for i in tw_mitaka_vader['text']:
    score = sentiment_scores(i)
    score_list.append(score)

In [168]:
tw_mitaka_vader['score'] = score_list

In [170]:
tw_mitaka_vader.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1285 entries, 0 to 1284
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   tweet_id    1285 non-null   int64              
 1   username    1285 non-null   object             
 2   text        1285 non-null   object             
 3   created_at  1285 non-null   datetime64[ns, UTC]
 4   location    827 non-null    object             
 5   source      1285 non-null   object             
 6   url         1285 non-null   object             
 7   score       1285 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 80.4+ KB


### Can see that most tweets are "negative" according to the algorithm, but I want to say that's mostly due to the language of the queried keywords in the first place.

In [171]:
print(tw_mitaka_vader.groupby('score').size())

score
Negative    758
Neutral     284
Positive    243
dtype: int64


In [40]:
print(tw_mitaka.groupby('source').size())

source
Apple refrigerator           1
Buffer                       1
Cheap Bots, Done Quick!     25
Coub                         1
Danbooru Bot                 7
IFTTT                        1
Instagram                    1
J1 Travel                    3
JRE_Chuo_E                   4
JRE_Sobu_E                   4
Otagakure News               1
Revive Social App           50
Tumblr                       2
TweetDeck                    9
Twitter                      2
Twitter Media Studio         1
Twitter Web App            309
Twitter for Android        414
Twitter for iPad            40
Twitter for iPhone         388
WeNano Spots                 2
WordPress.com                2
dlvr.it                      2
pixiv: Post to Twitter       1
twittbot.net                14
dtype: int64


In [176]:
tw_mitaka['location'].nunique()

572

In [ ]:
'''
Next Steps:
Create visualizations of findings.
Time, Source, Score, username maybe (who's tweeting the most?), location (need to clean)
'''